# Regresión espacial 

Consideraremos dos procesos relacionados pero muy diferentes que dan lugar a efectos espaciales: *Heterogeneidad Espacial (SH)* y *Dependencia Espacial (SD)*.

La heterogeneidad espacial se refiere a la variación en las propiedades de los datos en diferentes áreas del espacio, lo que implica que las relaciones entre las variables no necesariamente sean homogéneas en toda la región de estudio (Rey et al. 2023). La dependencia espacial, por otro lado, se refiere a la tendencia de valores similares a agruparse espacialmente (Anselin 1988). Esto implica que el valor de una variable en un punto del espacio está correlacionado con los valores de la misma variable en puntos cercanos, un fenómeno también conocido como autocorrelación. 

Cuando la heterogeneidad y la dependencia espacial no se incorporan adecuadamente en los modelos, la estructura espacial queda remanente en los residuales, lo cual se traduce en una falta de ajuste adecuada del modelo (Anselin 2022, Cressie 2015). Este remanente espacial en los residuos es una violación de las suposiciones de independencia que subyacen a los modelos clásicos, ya que los modelos clásicos suelen asumir que las observaciones son independientes entre sí (sin dependencia espacial) y que la relación entre las variables es constante en toda la región de estudio (sin heterogeneidad espacial) (Lesage 2011, Banerjee 2008). Esto afecta la validez de las inferencias estadísticas y resulta en modelos con predicciones sesgadas.



## Spatial models
Discrete data

1. Spatial heterogeneity:
    - Spatial Regimes (SR)
        * Spatial regimes slopes
        * Spatial fixed effects
    - Geographically Weighthed Regression (GWR)

2. Spatial dependency:
    - Conditional Autoregressive Models (CAR)
    - Simultaneous Autoregressive Models
        * Spatially-lagged X Model (SLX) [$WX_y$] - Autocorrelación exógena en los predictores
        * Spatially-lagged Y Model (SAR) [$\rho Wy$] - Autocorrelación endógena en la variable respuesta
        * Spatially-lagged Error Model (SEM) [$\lambda W \mu$]- Autocorrelación en el término del error


### Dependencia espacial


1. Modelos de regresión para dependencia espacial tipo SAR (Modelos autorregresivos espaciales)

Para incorporar la dependencia espacial en los modelos, se puede utilizar los llamados Modelos Autoregresivos Simultáneos (Whittle 1954), los cuales permiten modelar la dependencia espacial a través de la inclusión de una matriz de vecindad. Esta matriz define la estructura de los vecinos de cada unidad de mapeo, ya sea en función de la distancia entre unidades o a partir de relaciones de contigüidad (Anselin 1996, Elhorst 2014).

Los modelos de autoregresión espacial, en lugar de asumir que las observaciones son independientes entre sí, reconocen que las unidades cercanas en el espacio tienden a presentar valores similares (Anselin 1988`).

$$Y = \rho W Y + X \beta + \epsilon$$ 


2. Modelos de regresión para dependencia espacial tipo CAR (Modelos autorregresivos condicionales)

Rho típicamente varía entre 0 y 1 e indica la fuerza de la autocorrelación espacial. Un valor de 0 significa que no hay autocorrelación espacial, mientras que un valor más cercano a 1 indica una fuerte autocorrelación espacial.

A diferencia de los modelos CAR “propios” (donde rho < 1), la distribución conjunta de un modelo ICAR es “impropia”, lo que significa que no se integra a 1. Los modelos ICAR son particularmente útiles para modelar el suavizado espacial. Imponen una estructura donde las unidades vecinas tienen distribuciones posteriores que tienden a estar cerca unas de otras. Esto es útil cuando se espera que las áreas geográficamente cercanas tengan tasas o niveles de la variable de respuesta similares debido a factores espaciales no medidos. Los efectos espaciales en un modelo ICAR implican una restricción de suma a cero. Esto significa que la suma de los efectos espaciales a través de todas las unidades espaciales es cero. Al imponer la restricción de suma a cero, se fija un punto de referencia y se asegura que haya una única solución para los efectos espaciales que mejor se ajuste a los datos. 


### NOTA: 
Antes de usar modelos espaciales, se debe evaluar si existe estructura espacial en los datos de entrada o en el residuo de los modelos. Para ello, revisar el gráfico de dispersión de Moran y el estadístico $MI$ de Moran. El Índice de Moran (MI) varía de [1, -1]. Cero indica ausencia de autocorrelación espacial, +1 indica una agrupación perfecta de valores similares, y -1 indica dispersión.


# Análisis a nivel de Departamentos

In [3]:
import geopandas as gpd
import numpy as np
import libpysal
from libpysal.weights import Queen
from sklearn.preprocessing import StandardScaler
import spreg

# Cargar datos
gdf = gpd.read_file(r"G:\My Drive\Investigacion2025\Posgrado_Statistics\GeoAnalysis\data\Balso_Departamento.gpkg")
gdf.info()

# Renombramos variables
gdf = gdf.rename(columns={'NOMBRE_DPT': 'departamento', 
                    'AREA_OFICI': 'area',
                    'NUMPOINTS': 'conteo'})


<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 34 entries, 0 to 33
Data columns (total 15 columns):
 #   Column                                   Non-Null Count  Dtype   
---  ------                                   --------------  -----   
 0   ID_ESPACIA                               34 non-null     object  
 1   AREA_OFICI                               34 non-null     float64 
 2   NOMBRE_DPT                               34 non-null     object  
 3   NUMPOINTS                                34 non-null     float64 
 4   elev_mean                                34 non-null     float64 
 5   Temperatura_media_anual_mean             34 non-null     float64 
 6   Precipitacion_anual_mean                 34 non-null     float64 
 7   Rango_medio_diurno_mean                  34 non-null     float64 
 8   Precipitacion_mes_mas_lluvioso_mean      34 non-null     float64 
 9   Precipitacion_mes_mas_seco_mean          34 non-null     float64 
 10  Isotermalidad_mean              

In [4]:
# Renombrar variables en el gdf original a nombres cortos
gdf = gdf.rename(columns={
    'elev_mean': 'elev',
    'Temperatura_media_anual_mean': 'temp',
    'Precipitacion_anual_mean': 'precip',
    'Rango_medio_diurno_mean': 'rmd',
    'Precipitacion_mes_mas_lluvioso_mean': 'p_lluv',
    'Precipitacion_mes_mas_seco_mean': 'p_seco',
    'Isotermalidad_mean': 'isoterm',
    'Estacionalidad_de_la_temperatura_mean': 'est_temp',
    'Rango_anual_de_temperatura_mean': 'rango_temp',
    'Estacionalidad_de_la_precipitacion_mean': 'est_precip'
})

### 1. *Modelo SLX (Spatial Lag of X).* 

Autocorrelación exógena entre las variables predictoras

Evalua la siguiente pregunta: ¿Cómo la dependencia espacial de las variables explicativas influyen en la variable de interés? 

In [5]:

# Aplica la transformación log + 1 a la varaible de interes
import pandas as pd

gdf['y'] = np.log(gdf['conteo'] + 1)

# Definir las variables dependiente e independientes
y = gdf['y'].values.reshape((-1, 1))
# Ahora define la lista de variables independientes con los nombres cortos
independent_vars = ['elev', 'temp', 'precip', 'rmd', 'p_lluv',
    'p_seco', 'isoterm', 'est_temp', 'rango_temp', 'est_precip']
X = gdf[independent_vars].values

# Escalar las variables independientes
st = StandardScaler()
X_scaled = st.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=independent_vars)

# Crear la matriz de pesos espaciales por contigüidad tipo Queen
w = Queen.from_dataframe(gdf)

# Calcular la autocorrelación exógena de las variables independientes escaladas
wx_dict = {}
for var in independent_vars:
    wx_dict[f'w_{var}'] = libpysal.weights.spatial_lag.lag_spatial(w, X_scaled_df[var])

wx_df = pd.DataFrame(wx_dict)

# Combinar las variables independientes originales escaladas con sus versiones espacialmente rezagadas
slx_exog = pd.concat([X_scaled_df, wx_df], axis=1)

# Ajustar el modelo OLS con las variables exógenas espacialmente rezagadas (SLX)
ols_model_slx = spreg.OLS(gdf['y'].values.reshape((-1, 1)),
                          slx_exog.values,
                          name_y='log_conteo',
                          name_x=slx_exog.columns.tolist(),
                          name_w='queen_contiguity')

# Imprimir el resumen del modelo OLS con SLX
print(ols_model_slx.summary)

with open("resumen_modelo.txt", "w") as f:
    f.write(str(ols_model_slx.summary))

C:\Users\Valentina Cardona\AppData\Local\Temp\ipykernel_5440\1722801646.py:19: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  w = Queen.from_dataframe(gdf)


REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :        None
Dependent Variable  :  log_conteo                Number of Observations:          34
Mean dependent var  :      2.7670                Number of Variables   :          21
S.D. dependent var  :      1.9525                Degrees of Freedom    :          13
R-squared           :      0.8612
Adjusted R-squared  :      0.6476
Sum squared residual:     17.4641                F-statistic           :      4.0324
Sigma-square        :       1.343                Prob(F-statistic)     :    0.006512
S.E. of regression  :       1.159                Log likelihood        :     -36.918
Sigma-square ML     :       0.514                Akaike info criterion :     115.837
S.E of regression ML:      0.7167                Schwarz criterion     :     147.890

------------------------------------------------------------

c:\Users\Valentina Cardona\anaconda3\envs\GeoAnalysis\Lib\site-packages\libpysal\weights\contiguity.py:347: UserWarning: The weights matrix is not fully connected: 
 There are 2 disconnected components.
 There is 1 island with id: 25.
  W.__init__(self, neighbors, ids=ids, **kw)


### 2. Modelo de error espacial (Spatially-lagged Error Model - SEM) 

Autocorrelación endógena en el error

Modela la dependencia espacial en los errores del modelo, no en las variables predictoras ni en la variable dependiente.

In [6]:
# Ajustar el modelo de error espacial (GM_Error_Het)
spatial_error_model = spreg.GM_Error_Het(y, X_scaled, w=w,
                                      name_y='log_conteo',
                                      name_x=independent_vars,
                                      name_w='queen_contiguity')

# Imprimir el resumen del modelo de error espacial
print(spatial_error_model.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: GM SPATIALLY WEIGHTED LEAST SQUARES (HET)
------------------------------------------------------------
Data set            :     unknown
Weights matrix      :queen_contiguity
Dependent Variable  :  log_conteo                Number of Observations:          34
Mean dependent var  :      2.7670                Number of Variables   :          11
S.D. dependent var  :      1.9525                Degrees of Freedom    :          23
Pseudo R-squared    :      0.4716
N. of iterations    :           1                Step1c computed       :          No

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT         2.77805         0.21485        12.93021         0.00000
                elev        14.70040         3.55011

### 3. Modelo de retardo espacial (Spatially-lagged Y Model - SAR)

Autocorrelación endógena con la variable dependiente. Implica que el valor de (y) en una región depende de los valores de (y) en regiones vecinas.

In [7]:
# Ajustar el modelo de retardo espacial (GM_Lag) =SAR
spatial_lag_model = spreg.GM_Lag(y, X_scaled, w=w,
                                name_y='log_conteo',
                                name_x=independent_vars,
                                name_w='queen_contiguity')

# Imprimir el resumen del modelo de retardo espacial
print(spatial_lag_model.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: SPATIAL TWO STAGE LEAST SQUARES
--------------------------------------------------
Data set            :     unknown
Weights matrix      :queen_contiguity
Dependent Variable  :  log_conteo                Number of Observations:          34
Mean dependent var  :      2.7670                Number of Variables   :          12
S.D. dependent var  :      1.9525                Degrees of Freedom    :          22
Pseudo R-squared    :      0.4734
Spatial Pseudo R-squared:  0.4771

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT         2.48554         0.87660         2.83545         0.00458
                elev        13.30003         5.07197         2.62226         0.00873
                temp        13.51779  

# Análisis a nivel de Municipios

In [1]:
import geopandas as gpd
import numpy as np
import libpysal
from libpysal.weights import Queen
from sklearn.preprocessing import StandardScaler
import spreg

# Cargar datos
gdf = gpd.read_file(r"G:\My Drive\Investigacion2025\Posgrado_Statistics\GeoAnalysis\data\Balso_Municipios.gpkg")

gdf = gdf.rename(columns={'dpto_cnmbr': 'departamento', 
                    'mpio_cnmbr': 'municipio',
                    'mpio_narea': 'area',
                    'NUMPOINTS': 'conteo'})
gdf.head(3)

,departamento,municipio,area,conteo,elev_mean,Temperatura_media_anual_mean,Precipitacion_anual_mean,Rango_medio_diurno_mean,Precipitacion_mes_mas_lluvioso_mean,Precipitacion_mes_mas_seco_mean,Isotermalidad_mean,Estacionalidad_de_la_temperatura_mean,Rango_anual_de_temperatura_mean,Estacionalidad_de_la_precipitacion_mean,geometry
0,ANTIOQUIA,MEDELLÍN,374.834005,18.0,2030.00,18.157167,2487.400000,9.573711,335.133333,94.466667,89.996162,37.960908,10.638400,36.525846,"MULTIPOLYGON (((-75.66974 6.3736, -75.66965 6...."
1,ANTIOQUIA,ABEJORRAL,507.141095,0.0,1963.75,17.530000,2419.958333,8.855028,295.500000,102.708333,90.066731,30.902316,9.848834,31.626925,"MULTIPOLYGON (((-75.46938 5.94575, -75.46897 5..."
2,ANTIOQUIA,ABRIAQUÍ,296.894050,0.0,2375.80,15.494967,2391.933333,8.703756,329.600000,69.400000,87.985126,33.218638,9.897067,41.646924,"MULTIPOLYGON (((-76.08351 6.7505, -76.08325 6...."


In [2]:
# Renombrar variables en el gdf original a nombres cortos
gdf = gdf.rename(columns={
    'elev_mean': 'elev',
    'Temperatura_media_anual_mean': 'temp',
    'Precipitacion_anual_mean': 'precip',
    'Rango_medio_diurno_mean': 'rmd',
    'Precipitacion_mes_mas_lluvioso_mean': 'p_lluv',
    'Precipitacion_mes_mas_seco_mean': 'p_seco',
    'Isotermalidad_mean': 'isoterm',
    'Estacionalidad_de_la_temperatura_mean': 'est_temp',
    'Rango_anual_de_temperatura_mean': 'rango_temp',
    'Estacionalidad_de_la_precipitacion_mean': 'est_precip'
})

## Dependencia espacial
*Distribución normal (Regresión lineal Gaussiana)*

### 1. *Modelo SLX (Spatial Lag of X).* 

In [4]:
# Aplica la transformación log + 1 a la varaible de interes
import pandas as pd

gdf['y'] = np.log(gdf['conteo'] + 1)

# Definir las variables dependiente e independientes
y = gdf['y'].values.reshape((-1, 1))
# Ahora define la lista de variables independientes con los nombres cortos
independent_vars = ['elev', 'temp', 'precip', 'rmd', 'p_lluv',
    'p_seco', 'isoterm', 'est_temp', 'rango_temp', 'est_precip']
X = gdf[independent_vars].values

# Escalar las variables independientes
st = StandardScaler()
X_scaled = st.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=independent_vars)

# Crear la matriz de pesos espaciales por los 112 vecinos más cercanos. 10% del total de municipios
from pysal.lib import weights
w_112k = weights.distance.KNN.from_dataframe(gdf, k=112)

# Calcular la autocorrelación exógena de las variables independientes escaladas
wx_dict = {}
for var in independent_vars:
    wx_dict[f'w_{var}'] = libpysal.weights.spatial_lag.lag_spatial(w_112k, X_scaled_df[var])

wx_df = pd.DataFrame(wx_dict)

# Combinar las variables independientes originales escaladas con sus versiones espacialmente rezagadas
slx_exog = pd.concat([X_scaled_df, wx_df], axis=1)

# Ajustar el modelo OLS con las variables exógenas espacialmente rezagadas (SLX)
ols_model_slx = spreg.OLS(gdf['y'].values.reshape((-1, 1)),
                          slx_exog.values,
                          name_y='log_conteo',
                          name_x=slx_exog.columns.tolist(),
                          name_w='4_vecinos_cercanos')

# Imprimir el resumen del modelo OLS con SLX
print(ols_model_slx.summary)

with open("resumen_modelo.txt", "w") as f:
    f.write(str(ols_model_slx.summary))

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :        None
Dependent Variable  :  log_conteo                Number of Observations:        1121
Mean dependent var  :      0.3129                Number of Variables   :          21
S.D. dependent var  :      0.8498                Degrees of Freedom    :        1100
R-squared           :      0.1065
Adjusted R-squared  :      0.0903
Sum squared residual:     722.626                F-statistic           :      6.5575
Sigma-square        :       0.657                Prob(F-statistic)     :   3.497e-17
S.E. of regression  :       0.811                Log likelihood        :   -1344.523
Sigma-square ML     :       0.645                Akaike info criterion :    2731.046
S.E of regression ML:      0.8029                Schwarz criterion     :    2836.507

------------------------------------------------------------

### 2. Modelo de error espacial (Spatially-lagged Error Model - SEM) 

In [5]:
# Ajustar el modelo de error espacial (GM_Error_Het)
spatial_error_model = spreg.GM_Error_Het(y, X_scaled, w=w_112k,
                                      name_y='log_conteo',
                                      name_x=independent_vars,
                                      name_w='112_vecinos_cercanos')

# Imprimir el resumen del modelo de error espacial
print(spatial_error_model.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: GM SPATIALLY WEIGHTED LEAST SQUARES (HET)
------------------------------------------------------------
Data set            :     unknown
Weights matrix      :112_vecinos_cercanos
Dependent Variable  :  log_conteo                Number of Observations:        1121
Mean dependent var  :      0.3129                Number of Variables   :          11
S.D. dependent var  :      0.8498                Degrees of Freedom    :        1110
Pseudo R-squared    :      0.0714
N. of iterations    :           1                Step1c computed       :          No

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT         0.31294         0.02445        12.80081         0.00000
                elev         0.45003         0.3

### 3. Modelo de retardo espacial (Spatially-lagged Y Model - SAR)

In [6]:
# Ajustar el modelo de retardo espacial (GM_Lag) =SAR
spatial_lag_model = spreg.GM_Lag(y, X_scaled, w=w_112k,
                                name_y='log_conteo',
                                name_x=independent_vars,
                                name_w='112_vecinos_cercanos')

# Imprimir el resumen del modelo de retardo espacial
print(spatial_lag_model.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: SPATIAL TWO STAGE LEAST SQUARES
--------------------------------------------------
Data set            :     unknown
Weights matrix      :112_vecinos_cercanos
Dependent Variable  :  log_conteo                Number of Observations:        1121
Mean dependent var  :      0.3129                Number of Variables   :          12
S.D. dependent var  :      0.8498                Degrees of Freedom    :        1109
Pseudo R-squared    :      0.0703
Spatial Pseudo R-squared:  0.0316

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT         0.04505         0.07238         0.62243         0.53366
                elev        -0.10881         0.31618        -0.34414         0.73074
                temp        -0.092

# Otros intentos

### 1. *Modelo SLX (Spatial Lag of X).* 

In [11]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from pysal.lib import weights
import libpysal

# Crear la variable binaria 'evento'
gdf['evento'] = np.where(gdf['conteo'] >= 1, 1, 0)

# Definir variables independientes
independent_vars = ['elev', 'temp', 'precip', 'rmd', 'p_lluv',
                    'p_seco', 'isoterm', 'est_temp', 'rango_temp', 'est_precip']
X = gdf[independent_vars].values

# Escalar las variables independientes
st = StandardScaler()
X_scaled = st.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=independent_vars)

# Crear la matriz de pesos espaciales con los 4 vecinos más cercanos
w_4k = weights.distance.KNN.from_dataframe(gdf, k=4)

# Calcular autocorrelación espacial (lag) para cada variable independiente escalada
wx_dict = {}
for var in independent_vars:
    wx_dict[f'w_{var}'] = libpysal.weights.spatial_lag.lag_spatial(w_4k, X_scaled_df[var])

wx_df = pd.DataFrame(wx_dict)

# Combinar variables originales escaladas con sus versiones espacialmente rezagadas
slx_exog = pd.concat([X_scaled_df, wx_df], axis=1)

# Agregar constante para el modelo Logit
slx_exog_const = sm.add_constant(slx_exog)

# Ajustar el modelo Logit
logit_model = sm.Logit(gdf['evento'], slx_exog_const)
logit_result = logit_model.fit()

# Imprimir resumen
print(logit_result.summary())

# Guardar el resumen en un archivo
with open("resumen_modelo_logit.txt", "w") as f:
    f.write(str(logit_result.summary()))


Optimization terminated successfully.
         Current function value: 0.439012
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                 evento   No. Observations:                 1121
Model:                          Logit   Df Residuals:                     1100
Method:                           MLE   Df Model:                           20
Date:                Thu, 10 Jul 2025   Pseudo R-squ.:                  0.1222
Time:                        19:04:28   Log-Likelihood:                -492.13
converged:                       True   LL-Null:                       -560.67
Covariance Type:            nonrobust   LLR p-value:                 1.806e-19
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
const           -1.7613      0.105    -16.771      0.000      -1.967      -1.555
elev            -5.2166

In [12]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from pysal.lib import weights
import libpysal

# Variable dependiente binaria
gdf['evento'] = np.where(gdf['conteo'] >= 1, 1, 0)

# Crear matriz de pesos espaciales
w_4k = weights.distance.KNN.from_dataframe(gdf, k=4)

# Crear término espacialmente rezagado de la variable dependiente
gdf['w_evento'] = libpysal.weights.spatial_lag.lag_spatial(w_4k, gdf['evento'])

# Escalar variables independientes
independent_vars = ['elev', 'temp', 'precip', 'rmd', 'p_lluv',
                    'p_seco', 'isoterm', 'est_temp', 'rango_temp', 'est_precip']
X_scaled = StandardScaler().fit_transform(gdf[independent_vars])
X_scaled_df = pd.DataFrame(X_scaled, columns=independent_vars)

# Combinar X escalado con el término espacialmente rezagado
X_with_lag = pd.concat([X_scaled_df, gdf['w_evento']], axis=1)

# Agregar constante
X_const = sm.add_constant(X_with_lag)

# Ajustar modelo Logit
logit_model = sm.Logit(gdf['evento'], X_const)
logit_result = logit_model.fit()

# Mostrar resumen
print(logit_result.summary())


Optimization terminated successfully.
         Current function value: 0.418668
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                 evento   No. Observations:                 1121
Model:                          Logit   Df Residuals:                     1109
Method:                           MLE   Df Model:                           11
Date:                Thu, 10 Jul 2025   Pseudo R-squ.:                  0.1629
Time:                        19:13:31   Log-Likelihood:                -469.33
converged:                       True   LL-Null:                       -560.67
Covariance Type:            nonrobust   LLR p-value:                 2.848e-33
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.1908      0.124    -17.689      0.000      -2.434      -1.948
elev           0.5587      0.

# Heterogeneidad espacial

 Modelos que permiten la variación espacial de los coeficientes de las variables predictoras, permitiendo capturar la heterogeneidad inherente al terreno (Rey et al. 2023). Entre estos enfoques se encuentran los modelos multiniveles (Wong 1985) (también conocidos como jerárquicos, y en algunos casos modelos mixtos o regímenes espaciales) y los modelos de regresión espacial ponderada (GWR, por sus siglas en inglés) (Brunsdon 1996). 
 
 Los modelos multiniveles permiten modelar la variabilidad espacial mediante la introducción de estructuras jerárquicas. Resulta particularmente útil cuando los datos presentan una organización natural en grupos o niveles (por ejemplo, cuencas y subcuencas). En contraste, los modelos GWR son menos complejos computacionalmente, ya que no requieren definir niveles jerárquicos. Los modelos GWR permiten que los coeficientes de las variables predictoras varíen para cada observación.

Los efectos fijos espaciales introducen econometricamente la noción de heterogeneidad espacial. Lo hacen en la forma más simple posible: al permitir que el término constante varíe geográficamente. Los demás elementos de la regresión permanecen sin cambios y, por lo tanto, se aplican uniformemente en todo el espacio. La idea de los regímenes espaciales (RE) es generalizar el enfoque de los efectos fijos espaciales para permitir que no solo el término constante varíe, sino también cualquier otra variable explicativa. Esto implica que la ecuación que estaremos estimando es:
 $$\log{P_i} = \alpha_r + \sum_k \mathbf{X}_{ki}\beta_{k-r} + \epsilon_i$$

donde no solo permitimos que el término constante varíe por región ($\alpha_r$
), sino también cada otro parámetro ($\beta_{k-r}$).

Código en R. Se realizo para el dataset de Municipios con efecto aleatorios por departamento. Nota: Evalue los efectos aleatorio a nivel de municipios y se generaba una matriz singular. 

# Modelos con dependencia y hetereogeneidad espacial

Una característica fundamental de los datos espaciales es la presencia simultánea de dependencia espacial y heterogeneidad espacial. La dependencia espacial se refiere al hecho de que las observaciones cercanas tienden a parecerse más entre sí que las distantes, debido a procesos subyacentes que operan en el espacio geográfico. Por su parte, la heterogeneidad espacial se manifiesta en diferencias sistemáticas entre regiones o ubicaciones, ya sea por condiciones ambientales, sociales o geológicas que varían en el territorio. 

* Estrategias para modelar conjuntamente dependencia y heterogenidad espacial:
1. Los modelos autoregresivos condicionales (CAR), implementados en la librería INLA, permiten incorporar simultáneamente estos dos aspectos en un marco bayesiano. La dependencia espacial se modela mediante una matriz de vecindad, y la heterogeneidad se introduce agregando un efecto aleatorio no estructurado (iid), como en el modelo BYM (Besag-York-Mollié). 

2. En la familia de modelos simultáneos autoregresivos (SAR) —implementados en R mediante la librería spatialreg— la dependencia espacial puede incorporarse a través de la variable dependiente (modelo SARlag), de los errores (SARerror), o de las covariables (modelo SLX). En estos modelos, la heterogeneidad espacial se representa explícitamente mediante interacciones entre las covariables continuas y un factor espacial categórico, como subcuencas o regiones, permitiendo que los efectos de las variables explicativas cambien según la unidad espacial. Este tipo de interacción permite capturar variaciones estructurales (regímenes espaciales) sin necesidad de introducir efectos aleatorios, y es una forma eficaz de modelar heterogeneidad espacial en un contexto frecuentista.